In [ ]:
import pandas as pd
from ipywidgets import FileUpload, Button, VBox, Label
from IPython.display import display
from io import StringIO
from difflib import SequenceMatcher
import unicodedata

In [ ]:
# -----------------------
# Funções auxiliares
# -----------------------
def normalize_colname(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(" ", "_").replace("-", "_")
    return s

def find_column(cols, target_norm):
    for c in cols:
        if normalize_colname(c) == target_norm:
            return c
    return None

def safe_get_str(row, colname):
    val = row.get(colname, "")
    if pd.isna(val):
        return ""
    return str(val).strip()

def escolher_valor(antigo, novo, coluna):
    antigo_s = "" if pd.isna(antigo) else str(antigo).strip()
    novo_s = "" if pd.isna(novo) else str(novo).strip()

    if not antigo_s and not novo_s:
        return ""

    if "nome" in normalize_colname(coluna) and antigo_s and novo_s:
        similaridade = SequenceMatcher(None, antigo_s.lower(), novo_s.lower()).ratio()
        if similaridade < 0.5:
            return novo_s
        return novo_s if len(novo_s) >= len(antigo_s) else antigo_s

    if not novo_s:
        return antigo_s
    return novo_s

# -----------------------
# Interface
# -----------------------
label1 = Label("📄 Selecione a Planilha 1 (antiga):")
upload1 = FileUpload(accept='.csv', multiple=False)

label2 = Label("📄 Selecione a Planilha 2 (nova):")
upload2 = FileUpload(accept='.csv', multiple=False)

button = Button(description="🔄 Gerar Planilhas", button_style='success')
output_label = Label()

# -----------------------
# Processamento principal
# -----------------------
def process_files(_):
    if not upload1.value or not upload2.value:
        output_label.value = "⚠️ Selecione as duas planilhas antes de continuar!"
        return

    file1 = list(upload1.value.values())[0]
    file2 = list(upload2.value.values())[0]

    # 🔧 Ler planilhas automaticamente detectando o separador e respeitando aspas
    df1 = pd.read_csv(StringIO(file1['content'].decode('utf-8')),
                      dtype=str, keep_default_na=False,
                      sep=None, engine="python", quotechar='"')
    df2 = pd.read_csv(StringIO(file2['content'].decode('utf-8')),
                      dtype=str, keep_default_na=False,
                      sep=None, engine="python", quotechar='"')
    df1 = df1.fillna("")
    df2 = df2.fillna("")

    # Detectar colunas
    telefone_col = None
    for c in df1.columns:
        n = normalize_colname(c)
        if "tel" in n or "fone" in n or n == "telefone":
            telefone_col = c
            break
    if telefone_col is None:
        output_label.value = "❌ Nenhuma coluna de telefone encontrada!"
        return

    origem_col = find_column(df1.columns, "origem")
    obs_col = find_column(df1.columns, "observacao") or find_column(df1.columns, "observação")

    # Garantir coluna de observação
    if obs_col is None:
        df1["observação"] = ""
        df2["observação"] = ""
        obs_col = "observação"

    # -----------------------
    # 🔍 Identificar duplicatas antes de limpar
    # -----------------------
    df1[telefone_col] = df1[telefone_col].astype(str).str.replace(r'\D', '', regex=True)
    df2[telefone_col] = df2[telefone_col].astype(str).str.replace(r'\D', '', regex=True)
    df1[telefone_col] = df1[telefone_col].astype(str)
    df2[telefone_col] = df2[telefone_col].astype(str)

    dup1 = df1[df1[telefone_col].duplicated(keep=False)]
    dup2 = df2[df2[telefone_col].duplicated(keep=False)]
    duplicatas_para_revisao = pd.concat([dup1, dup2], ignore_index=True).drop_duplicates()

    # -----------------------
    # 🔍 Remover duplicados e registrar removidos
    # -----------------------
    removidos_duplic1 = df1[df1[telefone_col].duplicated(keep='last')]
    removidos_duplic2 = df2[df2[telefone_col].duplicated(keep='last')]
    removidos_por_duplicacao = pd.concat([removidos_duplic1, removidos_duplic2], ignore_index=True)

    df1 = df1.drop_duplicates(subset=[telefone_col], keep='last')
    df2 = df2.drop_duplicates(subset=[telefone_col], keep='last')

    # -----------------------
    # 🔄 Mesclar as duas planilhas
    # -----------------------
    merged = pd.merge(df1, df2, on=telefone_col, how='outer', suffixes=('_antigo', '_novo'))
    base_cols = [c for c in df1.columns if c != telefone_col]

    final_rows = []
    altered_rows = []

    for _, row in merged.iterrows():
        final_row = {}
        final_row[telefone_col] = row[telefone_col]

        obs_antiga = safe_get_str(row, f"{obs_col}_antigo")
        obs_nova = safe_get_str(row, f"{obs_col}_novo")
        observacao_final = obs_nova if obs_nova else obs_antiga

        houve_alteracao = False

        for col in base_cols:
            val_antigo = row.get(f"{col}_antigo", "")
            val_novo = row.get(f"{col}_novo", "")
            val_final = escolher_valor(val_antigo, val_novo, col)

            # Caso especial: mudança de origem → adicionar à observação
            if origem_col and normalize_colname(col) == normalize_colname(origem_col):
                origem_antiga = safe_get_str(row, f"{col}_antigo")
                origem_nova = safe_get_str(row, f"{col}_novo")
                if origem_antiga and origem_nova and origem_antiga != origem_nova:
                    complemento = f"entrada anterior: {origem_antiga}"
                    if observacao_final:
                        observacao_final = observacao_final.rstrip().rstrip(".") + ". " + complemento
                    else:
                        observacao_final = complemento

            final_row[col] = val_final

            antigo_str = safe_get_str(row, f"{col}_antigo")
            if antigo_str and val_final != antigo_str:
                houve_alteracao = True

        final_row[obs_col] = observacao_final
        final_rows.append(final_row)

        if houve_alteracao:
            altered_rows.append({
                telefone_col: row[telefone_col],
                **{f"{col}_antigo": safe_get_str(row, f"{col}_antigo") for col in base_cols},
                **{f"{col}_novo": final_row[col] for col in base_cols}
            })

    # -----------------------
    # 📊 Criar DataFrames finais
    # -----------------------
    final_df = pd.DataFrame(final_rows, columns=list(df1.columns))
    alterados_df = pd.DataFrame(altered_rows)
    duplicatas_para_revisao = duplicatas_para_revisao if not duplicatas_para_revisao.empty else pd.DataFrame(columns=df1.columns)
    removidos_por_duplicacao = removidos_por_duplicacao if not removidos_por_duplicacao.empty else pd.DataFrame(columns=df1.columns)

    # -----------------------
    # 🧾 Gerar relatório
    # -----------------------
    relatorio = f"""
📊 RELATÓRIO DE MUDANÇAS E DESAMBIGUAÇÃO
────────────────────────────────────────────
Planilha antiga: {len(df1)} registros
Planilha nova: {len(df2)} registros

🔁 Duplicatas encontradas: {len(duplicatas_para_revisao)}
🗑️ Removidos por duplicação: {len(removidos_por_duplicacao)}
✅ Registros finais (únicos): {len(final_df)}
✏️ Registros alterados: {len(alterados_df)}

📄 Arquivos gerados:
- planilha_final.csv
- planilha_alterados.csv
- duplicatas_para_revisao.csv
- removidos_por_duplicacao.csv
- relatorio_mudancas.txt
────────────────────────────────────────────
"""

    # -----------------------
    # 💾 Salvar arquivos com delimitador seguro
    # -----------------------
    final_df.to_csv("planilha_final.csv", index=False, sep=";", encoding="utf-8-sig", quotechar='"')
    alterados_df.to_csv("planilha_alterados.csv", index=False, sep=";", encoding="utf-8-sig", quotechar='"')
    duplicatas_para_revisao.to_csv("duplicatas_para_revisao.csv", index=False, sep=";", encoding="utf-8-sig", quotechar='"')
    removidos_por_duplicacao.to_csv("removidos_por_duplicacao.csv", index=False, sep=";", encoding="utf-8-sig", quotechar='"')

    with open("relatorio_mudancas.txt", "w", encoding="utf-8") as f:
        f.write(relatorio)

    output_label.value = (
        "✅ Arquivos gerados com sucesso!\n\n"
        "📁 planilha_final.csv — base consolidada (ponto e vírgula, UTF-8)\n"
        "📁 planilha_alterados.csv — registros alterados\n"
        "📁 duplicatas_para_revisao.csv — telefones repetidos\n"
        "📁 removidos_por_duplicacao.csv — duplicatas removidas\n"
        "📄 relatorio_mudancas.txt — resumo completo"
    )

# -----------------------
# Interface gráfica
# -----------------------
button.on_click(process_files)
display(VBox([label1, upload1, label2, upload2, button, output_label]))
